In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# from google.colab import files
# files.upload()
# !ls -lha kaggle.json
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d gargmanas/movierecommenderdataset

In [ ]:
# !unzip movierecommenderdataset.zip

# Data Understanding

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
print('jumlah data movies:   ', len(movies.movieId.unique()))
print('Jumlah data penilaian yang diberikan pengguna: ', len(ratings.userId.unique()))
print('Jumlah data penilaian movies: ', len(ratings.movieId.unique()))

## Movies

In [ ]:
movies.head()

In [ ]:
movies.info()

In [ ]:
print('jumlah data movies:   ', len(movies.movieId.unique()))
print('jumlah genre pada movies:   ',len(movies.genres.unique()))

In [ ]:
genres = movies.genres
stopwords = set(STOPWORDS)
comment_words = ''
for genre in genres:
 
    genre = str(genre)
    # split the different genres
    tokens = genre.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
wordcloud = WordCloud(width = 2000, height = 1000, background_color ='black', stopwords = stopwords, min_font_size = 10).generate(comment_words)
    
#Plot the wordcloud
plt.figure(figsize = (13, 10), facecolor = None)
plt.imshow(wordcloud)
plt.title("All Genre")
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

## Ratings

In [ ]:
ratings.head()

In [ ]:
ratings.info()

In [ ]:
ratings.shape

In [ ]:
ratings.describe()

In [ ]:
print('Jumlah user yang memberikan rating: ', len(ratings.userId.unique()))
print('Jumlah movies yang medapatkan rating dari user: ', len(ratings.movieId.unique()))
print('Jumlah data ratings: ', len(ratings))

In [ ]:
ratings = ratings.drop(['timestamp'], axis=1)
ratings

# Exploratory Data Analysis

In [ ]:
movies.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
sns.countplot(ratings.rating)

In [ ]:
new_movie = pd.merge(ratings, movies , on='movieId', how='left')
new_movie

In [ ]:
new_movie.isnull().sum()

## Buat Collaborative

In [ ]:
# Definisikan dataframe ratings ke dalam variabel all_movies_rate
all_movies_rate = ratings
all_movies_rate

In [ ]:
movie_df = new_movie.sort_values('movieId', ascending=True)
movie_df

In [ ]:
len(movie_df.movieId.unique())

In [ ]:
len(movie_df.title.unique())

In [ ]:
movie_df.genres.unique()
# print(len(fix_anime.genre.unique()))

In [ ]:
duplicate_rows_movies = movie_df[movie_df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_movies.shape)

In [ ]:
preparation = movie_df

In [ ]:
movie_id = preparation['movieId'].tolist()
 
movie_title = preparation['title'].tolist()
 
movie_genre = preparation['genres'].tolist()
 
print(len(movie_id))
print(len(movie_title))
print(len(movie_genre))

In [ ]:
movie_new = pd.DataFrame({
    'Id': movie_id,
    'Title': movie_title,
    'Genre': movie_genre
})
movie_new

In [ ]:
data = movie_new
data.sample(5)

In [ ]:
genres = []
for genre in data.Genre:
    genres = genres + [(genre.replace(' ', '_').replace('|', ' ').replace('-',''))]
pd.Series(genres).head(5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
 
# Melakukan perhitungan idf pada data genre
tf.fit(genres) 
 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names()

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(genres) 
 
# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

In [ ]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

In [ ]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan genre
# Baris diisi dengan nama anime
 
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names(),
    index=data.Title
).sample(20, axis=1).sample(10, axis=0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
 
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

In [ ]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa judul movie
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['Title'], columns=data['Title'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap resto
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

In [ ]:
def movie_recommendations(movie_title, similarity_data=cosine_sim_df, items=data[['Title', 'Genre']], k=7):
    """
    Rekomendasi movies berdasarkan kemiripan dataframe
 
    Parameter:
    ---
    movie_title : tipe data string (str)
                Nama anime (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan anime sebagai 
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---
 
 
    Pada index ini, kita mengambil k dengan nilai similarity terbesar 
    pada index matrix yang diberikan (i).
    """
 
 
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan    
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,movie_title].to_numpy().argpartition(
        range(-1, -k, -1))
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop nama_resto agar nama anime yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(movie_title, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
data[data.Title.eq('Toy Story (1995)')]

In [ ]:
# Mendapatkan rekomendasi restoran yang mirip dengan Naruto
movie_recommendations('Avengers: Age of Ultron (2015)')

# Collaborative Filtering

In [ ]:
# Membaca dataset
 
df = ratings
df

In [ ]:
# Mendapatkan jumlah user
num_users = len(user_to_user_encoded)
print(num_users)
 
# Mendapatkan jumlah anime
num_anime = len(anime_encoded_to_anime)
print(num_anime)
 
# Mengubah rating menjadi nilai float
df['rating'] = df['rating'].values.astype(np.float32)
 
# Nilai minimum rating
min_rating = min(df['rating'])
 
# Nilai maksimal rating
max_rating = max(df['rating'])
 
print('Number of User: {}, Number of anime: {}, Min Rating: {}, Max Rating: {}'.format(
    num_users, num_anime, min_rating, max_rating
))

In [ ]:
# Mengacak dataset
df = df.sample(frac=1, random_state=42)
df

In [ ]:
# Membuat variabel x untuk mencocokkan data user dan anime menjadi satu value
x = df[['user', 'anime']].values
 
# Membuat variabel y untuk membuat rating dari hasil 
y = df['rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
 
# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)
 
print(x, y)

In [ ]:
class RecommenderNet(tf.keras.Model):
 
  # Insialisasi fungsi
  def __init__(self, num_users, num_anime, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_anime = num_anime
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.anime_embedding = layers.Embedding( # layer embeddings anime
        num_anime,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.anime_bias = layers.Embedding(num_anime, 1) # layer embedding anime bias
 
  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) # memanggil layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) # memanggil layer embedding 2
    anime_vector = self.anime_embedding(inputs[:, 1]) # memanggil layer embedding 3
    anime_bias = self.anime_bias(inputs[:, 1]) # memanggil layer embedding 4
 
    dot_user_anime = tf.tensordot(user_vector, anime_vector, 2) 
 
    x = dot_user_anime + user_bias + anime_bias
    
    return tf.nn.sigmoid(x) # activation sigmoid

In [ ]:
model = RecommenderNet(num_users, num_anime, 50) # inisialisasi model
 
# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
# Memulai training
 
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 50000,
    epochs = 50,
    validation_data = (x_val, y_val)
)

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model_metrics')
plt.ylabel('root_mean_squared_error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
anime_df = anime_new
df = pd.read_csv('rating.csv')
 
# Mengambil sample user
user_id = df.user_id.sample(1).iloc[0]
anime_view_by_user = df[df.user_id == user_id]
 
anime_not_view = anime_df[~anime_df['id'].isin(anime_view_by_user.anime_id.values)]['id'] 
anime_not_view = list(
    set(anime_not_view)
    .intersection(set(anime_to_anime_encoded.keys()))
)
 
anime_not_view = [[anime_to_anime_encoded.get(x)] for x in anime_not_view]
user_encoder = user_to_user_encoded.get(user_id)
user_resto_array = np.hstack(
    ([[user_encoder]] * len(anime_not_view), anime_not_view)
)

In [ ]:
ratings = model.predict(user_resto_array).flatten()
 
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_anime_ids = [
    anime_encoded_to_anime.get(anime_not_view[x][0]) for x in top_ratings_indices
]
 
print('Showing recommendations for users: {}'.format(user_id))
print('===' * 9)
print('Animer with high ratings from user')
print('----' * 8)
 
top_anime_user = (
    anime_view_by_user.sort_values(
        by = 'rating',
        ascending=False
    )
    .head(5)
    .anime_id.values
)
 
anime_df_rows = anime_df[anime_df['id'].isin(top_anime_user)]
for row in anime_df_rows.itertuples():
    print(row.anime_name, ':', row.genre)
 
print('----' * 8)
print('Top 10 anime recommendation')
print('----' * 8)
 
recommended_anime = anime_df[anime_df['id'].isin(recommended_anime_ids)]
for row in recommended_anime.itertuples():
    print(row.anime_name, ':', row.genre)